# Inserting and editing records using the EMu REST API

This notebook provides examples of how to use this library to insert and edit records using the [EMu REST API](https://help.emu.axiell.com/emurestapi/latest/).

Fields that appear in grids in the EMu client are automatically grouped in the records returned by the API. The same groups must be used when inserting or editing data. xmu can automatically assign groups if a schema file is available.

In [ ]:
from xmu import EMuAPI, EMuRecord, move, remove, replace

Create an EMuAPI object by passing the path to a config file. An example config file is included in this directory. **Please make sure you are connecting to a test environment.**

In [ ]:
api = EMuAPI(config_path="../tests/emurestapi.toml")

To insert a record, first create the record as a `dict`. The xmu library uses the following conventions:

- Fields are identified by their backend name, including the Ref suffix for attachments. Backend names can be found in the field help in the client.
- Single-value fields can be entered as strings
- Tables are lists
- Nested tables are list of lists
- Attachments are represented as `dict`s following the same conventions

In [ ]:
rec = {
    "LocCountry": "United States",
    "LocProvinceStateTerritory": "Maine",
    "LocPreciseLocation": "Wales",
    "LatLatitude_nesttab": [["44°10′0″N"]],
    "LatLongitude_nesttab": [["70°3′54″W"]],
    "LatDatum_tab": ["WGS 84 (EPSG:4326)"],
    "ColDateVisitedFrom": "Jan 1970",
    "NteText0": ["API test record"],
    "NteAttributedToRef_nesttab": [[{"NamFirst": "Ima", "NamLast": "Test"}]],
}

It is generally fine to work with a `dict`, but behind the scenes xmu uses the `EMuRecord` class to ensure that records are well-formed and that data is in a form that EMu will recognize. We can see what that looks like by converting the above record:

In [ ]:
EMuRecord(rec, "ecollectionevents")

We can use the`insert()` method to insert a new record. This method (1) converts the record to the format needed for the API, (2) resolves any attachments, and (3) submits the request to create the record to the API. **The resolve function is currently too simple to use in production.** If successful, the newly created record is returned:

In [ ]:
resp = api.insert("ecollectionevents", rec)
test_rec = resp.first()
test_rec

Behind the scenes, the `dict` is converted to the format required using the `to_insert()` method. This method can be run directly to see the final record, including any groups:

In [ ]:
api.to_insert(rec, module="ecollectionevents")

## Editing existing records

Changes to existing records are handled using the `edit()` method. This method requires the user to specify a module, IRN, and patch. The patch is the list of changes to be made to the record using JSON Patch syntax, which differs signficantly from the format used to modify existing record using CSV or XML imports. Some salient differences include the following:

- Patch syntax consists of a list of changes. CSV/XML updates consist of records including all fields to be changed.
- Patches require users to group fields based on the groups defined in the schema. This requires knowledge of the schema and can result in complex paths. In CSV/XML updates, users can use the group attribute to explicitly group related fields, including fields that are not explcitly associated in the schema. The patch approach does not provide an obvious way to group related fields that are not part of the same grid in EMu (for example, the metadata fields associated with the coordinates table on the Lat/Long tab) and seems likely garble rows when inserting into related fields where not all fields are populated to the same level.
- Patch syntax allows users to replace or append data in tables, with the exact operation based on the path. CSV/XML updates use suffixes on field names to allow users to replace (1=), append (+), or prepend (-) rows.  
- When replacing rows in tables or groups, patches use a zero-based index to identify the row (so the first row is 0, the second row is 1, etc.) CSV/XML updates use a slightly modified one-based index (so the first row is either 0 or 1, the second row is 2, etc.)

One goal of the xmu library is to streamline the variations between the different ways to import or report data in EMu. To that end, xmu provides functions to convert the import syntax to patches that works for many (but not all) uses. It also includes five functions to define patch entries directly: 

- `add()` adds data to a field or a row to a table
- `remove()` clears a field or deletes a row from a table
- `replace()` replaces the contents of a field with a new value
- `move()` moves data from one field to another
- `test()` defines a test that the record must pass in order for the patch to be applied
 
To add a value to a single-value field, the patch can be supplied as a `dict` specifying the field name and value to add. Similar to `insert()`, the response to the `edit()` method includes the record as modified by the patch:

In [ ]:
patch = {"LocDistrictCountyShire": "Androscoggin Co."}
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
{k: v for k, v in resp.first().items() if k == "LocDistrictCountyShire"}

Behind the scenes, the `to_patch()` method is used to create the patch. This method can be run directly to see the final patch:

In [ ]:
api.to_patch(patch, module="ecollectionevents")

Alternatively, the patch can be viewed by looking at the body of the prepared request:

In [ ]:
resp.prepared.body

We can add a table in the same way:

In [ ]:
patch = {"LatGeoreferencingNotes0": ["Measured by GPS"]}
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
{k: v for k, v in resp.first().items() if k == "LatGeoreferencingNotes0"}

To remove a value, you can pass `None` to the key you wish to clear:

In [ ]:
patch = {"LocDistrictCountyShire": None}
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k == "LocDistrictCountyShire"}

Some operations don't lend themselves to this approach. For example, the API allows data to be moved between fields, that is, copied to a destination field and deleted from the source field. There is no obvious way to convey this with an EMuRecord object, so in this case the patch must be created manually:

In [ ]:
patch = [move(from_="LocPreciseLocation", path="LocTownship")]
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k in ["LocPreciseLocation", "LocTownship"]}

The suffixes used by CSV/XML imports can be used to append or replace data in table fields. For example, use the (+) suffix to append to a table:

In [ ]:
patch = {"LatDatum_tab(+)": ["NAD83 (EPSG:4269)"]}
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k in ["LatDatum_tab"]}

Or use a numeric suffix to replace data in a specific row:

In [ ]:
patch = {"LatDatum_tab(2=)": ["WGS 84 (EPSG:4326)"]}
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k in ["LatDatum_tab"]}

When working with tables, using a falsy value (like `None` but specifically excluding 0) will result in the table cell being replaced with an empty value, not removed:

In [ ]:
patch = {"LatDatum_tab(2=)": [None]}
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k in ["LatDatum_tab"]}

The `remove()` function must be used to remove a row:

In [ ]:
patch = [remove(path="/LatDatum_tab/1")]
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k in ["LatDatum_tab"]}

The same operations can generally be used for both nested tables (which allow multiple values in a single row cell) and groups (which include the fields that display together in a grid). One challenging part of the API is that it requires users to know the full path to the column they want to modify, which requires a detailed understanding of how EMu operates under the hood. This library can map those paths directly in many cases.

In [ ]:
patch = {
    "LatLatitude_nesttab(+)": [["44°10′0″N"]],
    "LatLongitude_nesttab(+)": [["70°3′54″W"]],
}
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k in ["LatComment_grp"]}

Some operations require creating the patch directly. For example, `EMuRecord` has pretty simplistic handling of nested tables. Basically, it can add or replace a single nested table but cannot modify the inner part of an existing nested table. The API offers a bit more flexibility here.

In [ ]:
patch = [
    replace(path="/LatComment_grp/1/LatComment_subgrp/0/LatLatitude", value="44 10 N")
]
resp = api.edit("ecollectionevents", test_rec["irn"], patch)
display(f"Patch: {resp.prepared.body}")
{k: v for k, v in resp.first().items() if k in ["LatComment_grp"]}

Determining the path to a field may be tricky, especially at first. To help with this, the EMuAPI object includes the `flatten()` method, which flattens the nested record structure to a one-level dictionary with keys corresponding to the path expected by the API:

In [ ]:
api.flatten("ecollectionevents", test_rec)